# Approximate Nearest Neighbors:

# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***
This is a modified version of the previous question and requires you to use an artificial nearest neighbors library

We suggest you to use one of the following:
- [ScaNN](https://github.com/google-research/google-research/tree/master/scann)
- [FAISS](https://github.com/facebookresearch/faiss)
- [Annoy](https://github.com/spotify/annoy.git)

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

You may use any other recommendation system that you wish to use. However,
evaluation script will score your submission by measuring the similarity between
users with the number of common images they appreciated.
The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [7]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from numpy import dot
from numpy.linalg import norm

In [ ]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [8]:
config = pd.read_csv('config.csv').iloc[0]
user = config['id']
k_value = config['k']

In [9]:
config

Unnamed: 0                            0
id                               276633
k                                     5
dataset_file    ./Behance_appreciate_1M
output_file                ./output.txt
Name: 0, dtype: object

### Read the Data

In [10]:
#with open("/content/Behance_subsampled.txt",'r') as inFile:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

In [11]:
print(len(appreciate_data))

1000000


### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [13]:
user_likes = dict()
for line in appreciate_data:
    line = line.strip()
    
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])

    if user_id not in user_likes:
        user_likes[user_id] = list()
    
    user_likes[user_id].append(item_id)

In [14]:
len(user_likes)

63497

In [15]:
#creating row,column,data for sparse matrix
row = []
column = []
data = []
for like in appreciate_data:
  like = like.strip()
  row.append(int(like.split()[0]))
  column.append(int(like.split()[1]))
  data.append(1)

In [16]:
#user and item mapping with index
#(user id, item id, unix timestamp)
user_tags = {}
index = 0
for user in row:
  if user not in user_tags.keys():
    user_tags[user] = index
    index += 1
print(len(user_tags))

item_tags = {}
index = 0
for item in column:
  if item not in item_tags.keys():
    item_tags[item] = index
    index += 1
print(len(item_tags))

63497
178788


In [17]:
users = []
for user in row:
  users.append(user_tags[user])
del row
items = []
for item in column:
  items.append(item_tags[item])
del column

In [18]:
#print(data)
print(len(items))
print(len(users))
print(len(data))
print(len(user_tags))
print(len(item_tags))

1000000
1000000
1000000
63497
178788


In [19]:
users = np.array(users, dtype = np.float32)
items = np.array(items,dtype = np.float32)
data = np.array(data,dtype = np.float32)
matrix = csr_matrix((data, (users, items)), shape=(len(user_tags), len(item_tags)))

##Unable to run below cell due to hardware constraints to select better n_components 

we can't compute for n_components greater than 4000 because of hardware limit.
If we increase n_components such that explained_variance_ratio.sum() gets to 0.85  to 0.90 we can see better results.

In [ ]:
def select_n_components(matrix,target_variance):
  """For selecting best number of n_components for capturing the variability of data well enough"""
  n_components_selected = 0
  while(True):
    n_components_selected += 200
    svd = TruncatedSVD(n_components = n_components_selected,random_state=23)
    svd.fit(matrix)
    if svd.explained_variance_ratio_.sum() >= target_variance :
      break
  return svd,n_components_selected

return_components = select_n_components(matrix,0.85)
print("Best Number for n_components:",return_components[1])
svd_matrix = return_components[0].fit_transform(matrix)

In [20]:
svd = TruncatedSVD(n_components=1000)
svd_matrix = svd.fit_transform(matrix)
print(svd.explained_variance_ratio_.sum())

0.4526052


### Use your choice of Approximate Nearest Neigbor after Collaborative Filtering to find nearest neighbors

###Scann

In [ ]:
!pip install scann
import scann

In [27]:
normalized_dataset = svd_matrix / np.linalg.norm(svd_matrix, axis=1)[:, np.newaxis]

In [29]:
def neighbors(user,k_value,normalized_dataset):
  """ returns an iterable object (like list or generator) """
  searcher = scann.scann_ops_pybind.builder(normalized_dataset, 20, "dot_product")
  #step 1: Partitioning
  searcher = searcher.tree(num_leaves=252, num_leaves_to_search=100, training_sample_size=len(normalized_dataset))
  #step2: Scoring
  searcher = searcher.score_ah(2, anisotropic_quantization_threshold=0.2).reorder(100).build()
  #step3: Re-Scoring
  neigh, distances = searcher.search(svd_matrix[user_tags[user]], final_num_neighbors=6)
  neighbors_scann = []
  for j in neigh:
    if j != user_tags[user]:    #Exclude user himself from the neighbors list
      for l in user_tags.keys():
        if user_tags[l] == j:
          neighbors_scann.append(l)
  #print(neighbor_tags)
  print("Neighbors:",neighbors_scann)
  return neighbors_scann

In [30]:
neigh = neighbors(user,k_value,normalized_dataset)

Neighbors: [1333087, 1815092, 2059570, 3613632, 938435]


### Open the output file to write all the lines to the file

In [32]:
outFile = open("output.txt", 'w')
liked_items = set(user_likes[user])
for n_user in neigh:
  for item_id in user_likes[n_user]:
    if item_id not in liked_items:
      outFile.write(str(item_id) + ' ' + str(n_user) + '\n')
      liked_items.add(item_id)
outFile.close()

### Answer the following questions:

#### Q1. **Explain how your choice of library works**

I am using ScaNN Library for implementation
> Working of the ScaNN Library:
*     Will be consisting of Three parts in implementation:
1.   Partitioning
>>*   It creates Partitions during training time. 
>>*   At query time it selects top partitions to find nearest neighbors in the total training datapoints in partitions for next step.
2.Scoring
>>*   Computes distance from the test-data point to all the datapoints in the top partitions selected in step-1
>>*   Scoring function returns nearest K' datapoints (K' > k) to the next step. 
3.Recoring:
>>*   From the best K' points returned from the previous step, re-computes these distances more accurately using dot-product.
>>*   We Normalized the dataset  before training, so dot-product will give us Cosine Similarity between datapoints.   
>>*   It returns K+1 nearest datapoints, In which test datapoint is also included.


#### Q2. **Compare your choice of library with vanilla KNN.**
```
Vanilla KNN Time Complexity: O(ndk)
ScaNN Library KNN: O(pdk)
    n: Number of datapoints in training dataset
    p: Partition size
    d: Dimensionality of datapoint
    k: Number of Neighbors
```
*   Even we use Asymetric hashing for distance calculation in top partitions made.
*   we only use actual distance measure for Rescoring part to find best K Neighbors out of K' neighbors returned in second step. 

#### Q3. **Compare your choice of library with implementation of ScaNN, faiss and annoy.**
*   In annoy it is mentioned it works best for datasets with dimensionality n some 100's and we can observe better results when dimensionality less than 100. But here we are using datasets of higher dimensionality.
*   Whereas In Scann, we can even use it for higher dimensionality dataset.  

####FAISS

In [ ]:
# pip install faiss-cpu --no-cache
# import faiss
# print(index.is_trained)
# index = faiss.IndexFlatL2(4000)
# index.add(matrix.toarray())
# test = list(svd_matrix[user_tags[user]])
# test1 = []
# test1.append(test)
# test_case = np.array(test1)
# test_case = np.array(test_case)
# test_case.shape
# D, I = index.search(test_case,5)
# neighbors = []
# for j in I[0]:
#   for l in user_tags.keys():
#     if user_tags[l] == j:
#       neighbors.append(l)
# print(I)
# print(neighbors)
# for item in neighbors:
#   print(user_likes[item])